In [2]:
import pandas as pd
import numpy as np

# Release wise stats test

In [3]:
count = 0
dfs = ['process','product','process+product']
orders = ["P", "C", "P+C"]
all_results = pd.DataFrame()
for k in range(len(dfs)):
    df = dfs[k]
    order = orders[k]
    result_df = pd.DataFrame()
    final_result = pd.read_pickle('results/Performance/commit_guru_file_specific/' + df +'_700_rf_release_devanvu_set3.pkl')
    for metric in final_result.keys():
        final_df = pd.DataFrame()
        release = [[],[],[]]
        for projects in final_result[metric].keys():
            if len(final_result[metric][projects]) < 3:
                continue
            count += 1
            i = 0
            for value in final_result[metric][projects]:
                if metric == 'ifa':
                    value = value/100
                release[i].append(value)
                i += 1
                if i == 3:
                    break
        for j in range(3):
            score_df = pd.DataFrame(release[j], columns = ['scores'])
            score_df['release'] = [j+1]*score_df.shape[0]
            final_df = pd.concat([final_df,score_df], axis = 0)
        final_df['metrics'] = [metric]*final_df.shape[0]
        result_df = pd.concat([result_df,final_df], axis = 0)
    result_df['Metric Type'] = [order]*result_df.shape[0]
    all_results = pd.concat([all_results,result_df])
all_results = all_results[all_results['metrics'] != 'featue_importance']
all_results = all_results.dropna()

In [14]:
for metric in all_results.metrics.unique():
    sub_df = all_results[all_results['metrics'] == metric]
    with open('results/Stats/Release_stability/' + metric + '.txt', 'w') as f:
        for mType in sub_df['Metric Type'].unique():
            sub_sub_df = sub_df[sub_df['Metric Type'] == mType]
            for rel in sub_sub_df.release.unique():
                f.write(mType + "_" + str(rel) + "\n")
                sub_sub_sub_df = sub_sub_df[sub_sub_df['release'] == rel]
                for score in sub_sub_sub_df.scores.values:
                    f.write("%s " % score)
                f.write("\n\n")
    f.close()

# correlation Stats Test

In [18]:
metrics = ['process','product','process+product']
exp_types = ['JIT','release']
all_results = pd.DataFrame()
for exp_type in exp_types:
    final_df = pd.DataFrame()
    for metric in metrics:
        project_corr = pd.read_pickle('results/Performance/Correlations/' + metric + '_corr_' + exp_type + '.pkl')
        flat_list = [np.nanmedian(sublist) for sublist in list(project_corr.values())]
        project_corr_df = pd.DataFrame(flat_list, columns = ['scores'])
        project_corr_df['metrics'] = [metric]*project_corr_df.shape[0]
        final_df = pd.concat([final_df,project_corr_df], axis = 0)
    final_df['type'] = [exp_type]*final_df.shape[0]
    all_results = pd.concat([all_results,final_df], axis = 0)
all_results = all_results.dropna()

/Users/suvodeepmajumder/anaconda3/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/suvodeepmajumder/anaconda3/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/suvodeepmajumder/anaconda3/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [21]:
with open('results/Stats/Correlations/file_corr.txt', 'w') as f:
    for metric in all_results.metrics.unique():
        sub_df = all_results[all_results['metrics'] == metric]
        for _type in sub_df.type.unique():
            sub_sub_df = sub_df[sub_df['type'] == _type]
            f.write(metric + "_" + _type + "\n")
            for score in sub_sub_df.scores.values:
                f.write("%s " % score)
            f.write("\n\n")
f.close()